
## Exercise 1 - Let's find corners

First let's implement harris corner deteciton. Please write a program that finds a corner of the following checker board image.
<img src="test/checkerboard.png"  style="width:200px"> 
The image is located at  <b>test/checkerboard.png</b> which is originally from https://github.com/hughesj919/HarrisCorner<br>
Type your code below.

Below is an example.

In [9]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import argparse
import operator

def findHarrisCorners(img, window_size, k, thresh):
    """
    Finds and returns list of corners and new image with corners drawn
    :param img: The original image
    :param window_size: The size (side length) of the sliding window
    :param k: Harris corner constant. Usually 0.04 - 0.06
    :param thresh: The threshold above which a corner is counted
    :return:  
    """
    
    #Find x and y derivatives
    dy, dx = np.gradient(img)
    Ixx = dx**2
    Ixy = dy*dx
    Iyy = dy**2
    height = img.shape[0]
    width = img.shape[1]

    cornerList = []
    newImg = Image.fromarray(np.uint8(img))  # prepare a color image for visualization
    color_img = Image.new("RGBA", newImg.size)
    color_img.paste(newImg)
    
    offset = int(window_size/2) # offset of the windowsize
    pixels = color_img.load() # a pointer to the pixel values

    #Loop through image and find our corners
    print("Finding Corners...")
    for y in range(offset, height-offset):
        for x in range(offset, width-offset):
            #Calculate sum of squares
            windowIxx = Ixx[y-offset:y+offset+1, x-offset:x+offset+1]
            windowIxy = Ixy[y-offset:y+offset+1, x-offset:x+offset+1]
            windowIyy = Iyy[y-offset:y+offset+1, x-offset:x+offset+1]
            Sxx = windowIxx.sum()
            Sxy = windowIxy.sum()
            Syy = windowIyy.sum()

            #Find determinant and trace, use to get corner response
            det = (Sxx * Syy) - (Sxy**2)
            trace = Sxx + Syy
            r = det - k*(trace**2)
            #if y%70 == 0 and x%50 == 0:
            #    print(r)

            #If corner response is over threshold, color the point and add to corner list
            if r > thresh:
                #print(x, y, r)
                cornerList.append([x, y, r])
                pixels[x,y] = (0,0,255)

    return color_img, cornerList


def findCorners(window_size, k, thresh, in_img_path, out_img_path):
        
    print("window size: " + str(window_size))
    print("number of corners: " + str(k))
    print("corner threshold: " + str(thresh))
    print("image file: " + in_img_path)        

    img = Image.open(in_img_path)
    
    if img is not None:
        gray= img.convert('L')
        pix = np.asarray(gray)
        print( "image size: " +str(pix.shape))    
       
        finalImg, cornerList = findHarrisCorners(pix, int(window_size), float(k), int(thresh))
        finalImg.save(out_img_path)

        # Write top 100 corners to file
        listsize = len(cornerList)
        cornerList.sort(key=operator.itemgetter(2))
        outfile = open('corners.txt', 'w')
        if listsize > 100:
            listsize = 100
        for i in range(listsize):
            outfile.write(str(cornerList[i][0]) + ' ' + str(cornerList[i][1]) + ' ' + str(cornerList[i][2]) + '\n')
        outfile.close()
    else:
        print("Cannot open"+in_img_path)
        
    print("done!")



Once your code is ready, let's run it.

In [10]:
win_size = 5 # Window size
k = 0.06 # k value for finding corners
thresh = 10000 
in_img_path = 'test/checkerboard.png'
out_img_path = 'harris_checker.png'

findCorners(win_size, k, thresh, in_img_path, out_img_path)

window size: 5
number of corners: 0.06
corner threshold: 10000
image file: test/checkerboard.png
image size: (384, 384)
Finding Corners...
done!


You will probably see an image like the one below.
<img src="test/checkerboard_harris.png"  style="width:300px"> 
But if you try with a more natural image, you would probably be dissappointed. The original images are from http://matthewalunbrown.com/autostitch/autostitch.html
<img src="test/mb/100-0023_img.jpg"  style="width:200px"> 

In [14]:
#in_img_path = 'test/pj/Rainier1.png'
in_img_path = 'test/mb/100-0023_img.jpg'
out_img_path = 'harris_mb.png'
findCorners(win_size, k, thresh, in_img_path, out_img_path)

window size: 5
number of corners: 0.06
corner threshold: 25000000000
image file: test/mb/100-0023_img.jpg
image size: (758, 568)
Finding Corners...
done!


You would see too many detected corners.
<img src="test/mb_harris.png"  style="width:200px"> 
Increasing the threshold makes better results, 

In [13]:
win_size = 5 # Window size
thresh = 25000000000
findCorners(win_size, k, thresh, in_img_path, out_img_path)

window size: 5
number of corners: 0.06
corner threshold: 25000000000
image file: test/pj/Rainier1.png
image size: (388, 517)
Finding Corners...
done!


but they still seem impractical.
<img src="test/mb_harris_2.png"  style="width:200px"> 
Can you think of improving the detection results? You might want to apply gaussian smoothing (it might be interesting to see that two 1-d convolutions is faster than one 2d convolution). It might be also good to apply non-maximum supression. But you will see a much better key-point locater in the next excercise. 